# Gensim 이용해 topic 추출


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install gensim

In [3]:
import pandas as pd
import numpy as np
from gensim import corpora, models

In [4]:
# DataFrame 불러오기
user_df = pd.read_csv('/content/drive/MyDrive/ASAC 5기/woowahan/dataset/final/encoded_user_df.csv')
biz_df = pd.read_csv('/content/drive/MyDrive/ASAC 5기/woowahan/dataset/final/encoded_biz_df_filtered.csv')
review_df = pd.read_csv('/content/drive/MyDrive/ASAC 5기/woowahan/dataset/final/encoded_review_df_filtered.csv')
active_area_df = pd.read_csv('/content/drive/MyDrive/ASAC 5기/woowahan/dataset/final/active_area_df.csv')

<ipython-input-4-3a181239334e>:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  user_df = pd.read_csv('/content/drive/MyDrive/ASAC 5기/woowahan/dataset/final/encoded_user_df.csv')


In [5]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987897 entries, 0 to 1987896
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             int64  
 1   name                object 
 2   yelping_since       object 
 3   useful              int64  
 4   funny               int64  
 5   cool                int64  
 6   elite               object 
 7   friends             object 
 8   fans                int64  
 9   compliment_hot      int64  
 10  compliment_more     int64  
 11  compliment_profile  int64  
 12  compliment_cute     int64  
 13  compliment_list     int64  
 14  compliment_note     int64  
 15  compliment_plain    int64  
 16  compliment_cool     int64  
 17  compliment_funny    int64  
 18  compliment_writer   int64  
 19  compliment_photos   int64  
 20  review_count_user   float64
 21  average_stars_user  float64
dtypes: float64(2), int64(16), object(4)
memory usage: 333.7+ MB


In [6]:
biz_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62586 entries, 0 to 62585
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   business_id        62586 non-null  int64  
 1   name               62586 non-null  object 
 2   address            61896 non-null  object 
 3   city               62586 non-null  object 
 4   state              62586 non-null  object 
 5   postal_code        62560 non-null  object 
 6   latitude           62586 non-null  float64
 7   longitude          62586 non-null  float64
 8   is_open            62586 non-null  int64  
 9   attributes         0 non-null      float64
 10  category           62586 non-null  object 
 11  region             62586 non-null  object 
 12  review_count_biz   62586 non-null  float64
 13  average_stars_biz  53450 non-null  float64
dtypes: float64(5), int64(2), object(7)
memory usage: 6.7+ MB


In [7]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2680161 entries, 0 to 2680160
Data columns (total 10 columns):
 #   Column       Dtype  
---  ------       -----  
 0   review_id    int64  
 1   user_id      int64  
 2   business_id  int64  
 3   stars        float64
 4   useful       int64  
 5   funny        int64  
 6   cool         int64  
 7   text         object 
 8   date         object 
 9   text_length  int64  
dtypes: float64(1), int64(7), object(2)
memory usage: 204.5+ MB


In [8]:
active_area_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 974954 entries, 0 to 974953
Data columns (total 3 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   user_id               974954 non-null  int64 
 1   most_visited_regions  974954 non-null  object
 2   visit_count           974954 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 22.3+ MB


## 1. install library

In [9]:
import nltk
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
# 문장으로 분리 토크나이저 정의
tokenizer = PunktSentenceTokenizer()


## 2. 문장분리


In [20]:
doc=review_df.loc[2]['text']

In [21]:
sentences = tokenizer.tokenize(doc)
sentences

['I was really between 3 and 4 stars for this one.',
 'I LOVE the 96th street Naked Tchopstix so I was very excited to see this one which is closer to my house.',
 'The vibe is totally different as this is geared more to take out although they do have a decent size dining area.',
 'You order at the counter and they deliver it.',
 'My daughter and I tried the sushi bowl.',
 'You pick up a piece of paper and  select from a choice of proteins, vegetables, rice and toppings.',
 "I like the fact that it is on paper and you don't have to guess what things are or have to shout over the counter to tell the server what you want.",
 'The sushi bowl was good but not wonderful.',
 'The choices particularly of the vegetables seemed a little too limiting and I felt I had more rice than other food items.',
 'The sauces were tasty.',
 'My husband ordered the Maui roll which is my favorite roll at the 96th street location and this just was not up that standard.',
 'It was good but the quality and consi

## 3. 토픽 추출

In [22]:
# 각 문장을 토큰화하여 전처리 (이 경우 추가 전처리는 하지 않음)
processed_sentences = [sentence.split() for sentence in sentences]

In [23]:
# 사전과 코퍼스 생성
dictionary = corpora.Dictionary(processed_sentences)
corpus = [dictionary.doc2bow(text) for text in processed_sentences]

In [24]:
# LDA 모델 구성, 여기서는 토픽 수를 5로 설정
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)


In [25]:
# 토픽 출력
topics = lda_model.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.038*"it" + 0.038*"I" + 0.038*"a" + 0.038*"sushi"')
(1, '0.040*"I" + 0.040*"to" + 0.032*"is" + 0.024*"the"')
(2, '0.062*"was" + 0.047*"good" + 0.032*"the" + 0.032*"not"')
(3, '0.076*"the" + 0.039*"to" + 0.024*"have" + 0.024*"is"')
(4, '0.060*"and" + 0.031*"of" + 0.031*"a" + 0.017*"You"')
